# Train and Deploy your first model in the cloud using Amazon SageMaker

While many data science experiments start with exploring data and training a first model on a notebook instance, at some point these experiments need to be operationalized to actually provide business value.
This is where Amazon SageMaker can help which allows to fully automate the whole machine learning lifecycle.

Similar to the previous lab we will now train our first model. However we will do the training on infrastructure running in AWS using a simple API call and then create an endpoint which can be used to serve predictions. We will use the Amazon SageMaker built-in XGBoost support to train and serve our model. 

In [11]:
import pandas as pd
import sagemaker      # to work with sagemaker
import boto3          # AWS SDK e.g. to uplaod to S3
import os

# let's use the sagemaker default bucket to store data
bucket=sagemaker.Session().default_bucket()
# the path in the s3 bucket whre to store artifacts
prefix = "marketing-xgboost"
# get the default sagemaker role which will be used to run the training job
role = sagemaker.get_execution_role()

# Data preparation

Amazon SageMaker XGBoost can train on data in either as CSV or LibSVM format. For this example, we’ll stick with CSV. It should have the predictor variable in the first column and NOT have a header row. The data then needs to be uploaded to S3 so it can be used during Training in Amazon SageMaker. 

In [12]:
# load the datasets
train = pd.read_csv("data/train.csv")
validation = pd.read_csv("data/validation.csv")
test = pd.read_csv("data/test.csv")

In [13]:
# Put target variable column (y_yes) at the front and write final CSV
first_column = train.pop('y_yes')
train.insert(0, 'y_yes', first_column)
train.to_csv('train_final.csv', index=False, header=False)

first_column = validation.pop('y_yes')
validation.insert(0, 'y_yes', first_column)
validation.to_csv('validation_final.csv', index=False, header=False)

first_column = test.pop('y_yes')
test.insert(0, 'y_yes', first_column)
test.to_csv('test_final.csv', index=False, header=False)

Now we'll copy the data to S3 for Amazon SageMaker's managed training job to pickup. We will only use training and validation dataset in our training job. 

In [14]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train_final.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation_final.csv')

First we'll need to specify which algorithm container to use. The sagemaker API has an API call to retrieve the location of the latest container images in ECR. Here we request to fetch the container URL for XGBoost. 

In [15]:
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

In [16]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

In SageMaker you will use and Estimator to train your machine learning model. When defining an estimator you need to specify a variety of training parameters.  This includes:
1. The `xgboost` algorithm container
1. The IAM role to use
1. Training instance type and count
1. S3 location for output data
1. Algorithm hyperparameters

And then a `.fit()` function which specifies:
1. S3 location for output data.  In this case we have both a training and validation set which are passed in.

In [17]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

# Notice the set of hyperparameters which we are setting. 
# The key one is objective, which specifies "binary:logistic" as we have a binary classifation task here. 
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

# Fit starts the training process, we need to point to the input data in S3 here
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

2022-07-04 09:19:06 Starting - Starting the training job...ProfilerReport-1656926345: InProgress
...
2022-07-04 09:19:43 Starting - Preparing the instances for training.........
2022-07-04 09:21:25 Downloading - Downloading input data...
2022-07-04 09:22:05 Training - Downloading the training image......
2022-07-04 09:23:05 Training - Training image download completed. Training in progress..Arguments: train
[2022-07-04:09:23:07:INFO] Running standalone xgboost training.
[2022-07-04:09:23:07:INFO] File size need to be processed in the node: 4.35mb. Available memory size in the node: 8473.68mb
[2022-07-04:09:23:07:INFO] Determined delimiter of CSV input is ','
[09:23:07] S3DistributionType set as FullyReplicated
[09:23:07] 28831x59 matrix with 1701029 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-07-04:09:23:07:INFO] Determined delimiter of CSV input is ','
[09:23:07] S3DistributionType set as FullyReplicated
[09:23:07] 8279x59 matrix with 48846

---

## Hosting
Now that we've trained the `xgboost` algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [18]:
# cell 18
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

-------!

---

## Evaluation
There are many ways to compare the performance of a machine learning model, but let's start by simply comparing actual to predicted values.  In this case, we're simply predicting whether the customer subscribed to a term deposit (`1`) or not (`0`), which produces a simple confusion matrix.

First we'll need to determine how we pass data into and receive data from our endpoint.  Our data is currently stored as NumPy arrays in memory of our notebook instance.  To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

*Note: For inference with CSV format, SageMaker XGBoost requires that the data does NOT include the target variable.*

In [19]:
# cell 19
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [20]:
import numpy as np

def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(validation.drop([ 'y_yes'], axis=1).to_numpy(), xgb_predictor)
predictions

array([0.07057323, 0.04853175, 0.0980761 , ..., 0.03528015, 0.07282169,
       0.02169198])

Now we'll check our confusion matrix to see how well we predicted versus actuals.

In [21]:
# cell 21
pd.crosstab(index=validation['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,7238,145
1,686,210


So, of the ~4000 potential customers, we predicted 136 would subscribe and 94 of them actually did.  We also had 389 subscribers who subscribed that we did not predict would.  This is less than desirable, but the model can (and should) be tuned to improve this.  Most importantly, note that with minimal effort, our model produced accuracies similar to those published [here](https://core.ac.uk/download/pdf/55631291.pdf).

_Note that because there is some element of randomness in the algorithm's subsample, your results may differ slightly from the text written above._

---

## Extensions

This example analyzed a relatively small dataset, but utilized Amazon SageMaker features such as distributed, managed training and real-time model hosting, which could easily be applied to much larger problems.  In order to improve predictive accuracy further, we could tweak value we threshold our predictions at to alter the mix of false-positives and false-negatives, or we could explore techniques like hyperparameter tuning.  In a real-world scenario, we would also spend more time engineering features by hand and would likely look for additional datasets to include which contain customer information not available in our initial dataset.



## End of Lab 3

---

## Automatic model Tuning (optional)
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose.
For example, suppose that you want to solve a binary classification problem on this marketing dataset. Your goal is to maximize the area under the curve (auc) metric of the algorithm by training an XGBoost Algorithm model. You don't know which values of the eta, alpha, min_child_weight, and max_depth hyperparameters to use to train the best model. To find the best values for these hyperparameters, you can specify ranges of values that Amazon SageMaker hyperparameter tuning searches to find the combination of values that results in the training job that performs the best as measured by the objective metric that you chose. Hyperparameter tuning launches training jobs that use hyperparameter values in the ranges that you specified, and returns the training job with highest auc.


In [22]:
# cell 22
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                            'min_child_weight': ContinuousParameter(1, 10),
                            'alpha': ContinuousParameter(0, 2),
                            'max_depth': IntegerParameter(1, 10)}


In [23]:
# cell 23
objective_metric_name = 'validation:auc'

In [24]:
# cell 24
tuner = HyperparameterTuner(xgb,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=9,
                            max_parallel_jobs=3)


In [25]:
# Not this can take about 20 minutes as it is executing 9 training jobs with 3 in parallel
tuner.fit({'train': s3_input_train, 'validation': s3_input_validation})

....................................................................................................................................................................................................................................................................................................................................................................!


In [26]:
boto3.client('sagemaker').describe_hyper_parameter_tuning_job(
HyperParameterTuningJobName=tuner.latest_tuning_job.job_name)['HyperParameterTuningJobStatus']

'Completed'

In [27]:
# return the best training job name
tuner.best_training_job()

'xgboost-220704-0927-018-31d05b9c'

In [28]:
#  Deploy the best trained or user specified model to an Amazon SageMaker endpoint
tuner_predictor = tuner.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')


2022-07-04 09:52:32 Starting - Preparing the instances for training
2022-07-04 09:52:32 Downloading - Downloading input data
2022-07-04 09:52:32 Training - Training image download completed. Training in progress.
2022-07-04 09:52:32 Uploading - Uploading generated training model
2022-07-04 09:52:32 Completed - Training job completed
------!

In [29]:
# Create a serializer
tuner_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [30]:
# Predict
predictions = predict(test.drop([ 'y_yes'], axis=1).to_numpy(),tuner_predictor)

In [31]:
# Collect predictions and convert from the CSV output our model provides into a NumPy array
pd.crosstab(index=test['y_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,3584,65
1,326,103


### (Optional) Clean-up

If you are done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [32]:
# cell 32
xgb_predictor.delete_endpoint(delete_endpoint_config=True)

In [33]:
# cell 33
tuner_predictor.delete_endpoint(delete_endpoint_config=True)